In [1]:
from misc import *

from sklearn.preprocessing import LabelEncoder
from scipy.special import logit
from theano import shared

import pymc3 as pm
import pandas as pd
import numpy as np


WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\Anaconda3\envs\MSR\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "C:\Anaconda3\envs\MSR\lib\site-packages\theano\gpuarray\__init__.py", line 227, in <module>
    use(config.device)
  File "C:\Anaconda3\envs\MSR\lib\

In [2]:
#Load the survey data

index1 = preprocess(pd.read_excel('data/RawData/IN10001.xls'))
index4 = preprocess(pd.read_excel('data/RawData/IN10004.xls'))
index5 = preprocess(pd.read_excel('data/RawData/IN10005.xls'))
index6 = preprocess(pd.read_excel('data/RawData/IN10006.xls'))
index7 = preprocess(pd.read_excel('data/RawData/IN10007.xls'))

#Create a new dataframe that only contains the demographic information that's included in each survey

common_columns = list(set(index1.columns).intersection(set(index7.columns)))

common_data = pd.concat([index1[common_columns],
                         index4[common_columns],
                         index5[common_columns],
                         index6[common_columns],
                         index7[common_columns]
                        ])

#Drop the columns that don't contain demographic information

common_data.drop(['ID', 'ADID IDFA', 'Time Started', 'Time Finished'], axis=1, inplace=True)

#Get count of respondents by state

common_data.groupby(['Area']).size().reset_index().values


array([['Alabama', 122],
       ['Alaska', 3],
       ['Arizona', 110],
       ['Arkansas', 37],
       ['California', 536],
       ['Colorado', 52],
       ['Connecticut', 44],
       ['Delaware', 9],
       ['District of Columbia', 33],
       ['Florida', 380],
       ['Georgia', 196],
       ['Hawaii', 10],
       ['Idaho', 22],
       ['Illinois', 286],
       ['Indiana', 105],
       ['Iowa', 59],
       ['Kansas', 36],
       ['Kentucky', 154],
       ['Louisiana', 72],
       ['Maine', 23],
       ['Maryland', 112],
       ['Massachusetts', 68],
       ['Michigan', 279],
       ['Minnesota', 70],
       ['Mississippi', 47],
       ['Missouri', 111],
       ['Montana', 9],
       ['Nebraska', 39],
       ['Nevada', 36],
       ['New Hampshire', 14],
       ['New Jersey', 96],
       ['New Mexico', 13],
       ['New York', 305],
       ['North Carolina', 220],
       ['North Dakota', 4],
       ['Ohio', 331],
       ['Oklahoma', 77],
       ['Oregon', 57],
       ['Pennsylvania', 

In [3]:
#Encode whether or not a respodent will vote in the 2018 House of Representatives election

def will_vote(answer):
    if answer == "Won't Vote":
        return 0
    else:
        return 1

#Encode whether or not a respondent will vote for a major party candidate given that he/she will be voting
    
def will_vote_major(answer):
    if answer == 'Will vote Republican' or answer == 'Will vote Democratic':
        return 1
    else:
        return 0

#Encode which major party candidate a respondent will vote for given that he/she will be voting for a major party candidate
    
def which_major(answer):
    if answer == 'Will vote Republican':
        return 0
    elif answer == 'Will vote Democratic':
        return 1
    
def encode_demo(index):
    #Create separate LabelEncoder instances for each variable so that they can be used to perform inverse transformations on the results later

    gender_enc = LabelEncoder()
    race_enc = LabelEncoder()
    age_enc = LabelEncoder()
    area_enc = LabelEncoder()
    edu_enc = LabelEncoder()
    div_enc = LabelEncoder()

    race_gender_enc = LabelEncoder()
    age_edu_enc = LabelEncoder()
    age_gender_enc = LabelEncoder()
    edu_gender_enc = LabelEncoder()

    #Encode the gender, race, age, education, and state categories as integers

    index['Gender Encoded'] = gender_enc.fit_transform(index['Gender'])
    index['Race Encoded'] = race_enc.fit_transform(index['Race'])
    index['Age Encoded'] = age_enc.fit_transform(index['Age'])
    index['Area Encoded'] = area_enc.fit_transform(index['Area'])
    index['Education Encoded'] = edu_enc.fit_transform(index['Education'])
    index['Division Encoded'] = div_enc.fit_transform(index['US Census Division'])

    #Create new columns for the marginal categories

    index['Race_Gender'] = index['Race'].str.cat(index['Gender'], sep='|')
    index['Age_Education'] = index['Age'].str.cat(index['Education'], sep='|')
    index['Age_Gender'] = index['Age'].str.cat(index['Gender'], sep='|')
    index['Education_Gender'] = index['Education'].str.cat(index['Gender'], sep='|')

    #Encode the marginal categories as integers

    index['Race_Gender'] = race_gender_enc.fit_transform(index['Race_Gender'])
    index['Age_Education'] = age_edu_enc.fit_transform(index['Age_Education'])
    index['Age_Gender'] = age_gender_enc.fit_transform(index['Age_Gender'])
    index['Education_Gender'] = edu_gender_enc.fit_transform(index['Education_Gender'])

    return index


In [4]:
index1['Will Vote'] = index1['Who will you vote for in the House of Representatives in 2018?'].apply(lambda row: will_vote(row))
index1['Will Vote Major'] = index1['Who will you vote for in the House of Representatives in 2018?'].apply(lambda row: will_vote_major(row))
index1['Which Major'] = index1['Who will you vote for in the House of Representatives in 2018?'].apply(lambda row: which_major(row))

common_data = encode_demo(common_data)
index1 = encode_demo(index1)


In [5]:
#Group by race, gender, education, and age to see how many are in each cell and how many in each cell will vote

index1_unique = index1.groupby(['Race Encoded',
                                'Gender Encoded',
                                'Education Encoded',
                                'Age Encoded',
                                'Area Encoded',
                                'Division Encoded',
                                'Race_Gender',
                                'Age_Education',
                                'Age_Gender',
                                'Education_Gender'])['Will Vote']
index1_unique = index1_unique.agg([('Will Vote', 'sum'), ('n', 'size')]).reset_index()


In [6]:
#Obtain a mapping from each state to its census division

division_map = common_data.groupby(['Area', 'US Census Division', 'Area Encoded', 'Division Encoded']).size().reset_index()[['Area', 'US Census Division', 'Area Encoded', 'Division Encoded']]

#Remove duplicate state rows where the census division is missing

division_map = division_map.loc[division_map['US Census Division'] != 'Unknown'].reset_index(drop=True)


In [7]:
#Load the 2016 Presidential Election results by state

state_df = pd.read_csv('demographics/US Presidential Results & PVIs by State 1828-2016 - 2-Party US Pres Results & PVIs.csv',
                       header=1, usecols=[0, 2])
state_df.columns = ['Area', 'Trump Vote']

#Use the same name for Washington D.C. in both dataframes

state_df.loc[state_df['Area'] == 'Washington DC', 'Area'] = 'District of Columbia'


#Join the 2016 Election results with the census divisions

state_df = state_df.merge(division_map)

#Calculate the log odds of the proportion of each state's voters that voted for Donald Trump in 2016

state_trump = logit(state_df['Trump Vote'].values/100)


In [8]:
#Create Theano constant variables for the demographic and marginal demographic variables

age = shared(index1_unique['Age Encoded'].values)
edu = shared(index1_unique['Education Encoded'].values)
race = shared(index1_unique['Race Encoded'].values)
gender = shared(index1_unique['Gender Encoded'].values)
area = shared(index1_unique['Area Encoded'].values)
div = shared(index1_unique['Division Encoded'].values)

age_gender = shared(index1_unique['Age_Gender'].values)
race_gender = shared(index1_unique['Race_Gender'].values)
age_edu = shared(index1_unique['Age_Education'].values)
edu_gender = shared(index1_unique['Education_Gender'].values)

#Create a Theano constant variable for the number of observations in each population cell

n = shared(index1_unique['n'].values)


In [9]:
#Define the model for the state-level effects coefficient

def hierarchical_normal(name, shape, mu=0):
    delta = pm.Normal('delta_{}'.format(name), 0, 1, shape=shape)
    sigma = pm.HalfCauchy('sigma_{}'.format(name), 5)
    
    return pm. Deterministic(name, mu + delta*sigma)


In [10]:
"""
This section creates the state-level variables.
===============================================

The length of the alpha_division coefficient vector is
the number of unique census divisions.
The length of the alpha_state coefficient vector is the
number of unique state values, excluding "Unknown".

The alpha_division and alpha_state variables have a mean of 0 and
a standard deviation of delta*sigma, where delta is sampled from a
normal distribution with a mean of 0 and standard deviation of 1, and
sigma is sampled from a half Cauchy distribution with beta = 5.

The beta_trump variable is the intercept for the support for Trump in
each state in the 2016 election. This variable is normally distributed
with a mean of 0 and a standard deviation of 5.b

The mu_state vector represents the mean for each of the 50 states
and Washington DC.
The state_trump vector is the log odds of support for Trump in each state
in the 2016 election.
"""

with pm.Model() as model:
    alpha_division = hierarchical_normal('region',
                                       common_data['Division Encoded'].unique().size)
    beta_trump = pm.Normal('trump', 0. ,5)
    mu_state = alpha_division[division_map['Division Encoded'].values] + beta_trump*state_trump
    alpha_state = hierarchical_normal('state',
                                      common_data['Area Encoded'].unique().size - 1,
                                      mu=mu_state)


In [11]:
"""
This section creates the general population variables.
======================================================

The intercept beta_0 uses an initial value for sampling that's equal to
the log odds of a respondent voting based on the survey responses.

Each of the general population demographic and marginal demographic
variables have a mean of 0 and a standard deviation of delta*sigma,
where delta is sampled from a normal distribution with a mean of 0
and standard deviation of 1, and sigma is sampled from a
half Cauchy distribution with beta = 5.

The variable eta is the multilevel logistic regression model that models
the log odds of the voter turnout rate.

"""

with model:
    beta_0 = pm.Normal('beta_0', 0, 5,
                       testval=logit(index1['Will Vote'].mean()))
    
    alpha_race_gender = hierarchical_normal('race_gender',
                                            index1['Race_Gender'].unique().size)
    alpha_age_gender = hierarchical_normal('age_gender',
                                           index1['Age_Gender'].unique().size)
    alpha_age_edu = hierarchical_normal('age_education',
                                        index1['Age_Education'].unique().size)
    alpha_edu_gender = hierarchical_normal('education_gender',
                                           index1['Education_Gender'].unique().size)
    
    alpha_age = hierarchical_normal('age',
                                    index1['Age'].unique().size)
    alpha_edu = hierarchical_normal('education',
                                    index1['Education'].unique().size)
    alpha_race = hierarchical_normal('race',
                                     index1['Race'].unique().size)
    alpha_gender = hierarchical_normal('gender',
                                       index1['Gender'].unique().size)
    alpha_area = hierarchical_normal('area',
                                     index1['Area'].unique().size)
    alpha_div = hierarchical_normal('division',
                                    index1['US Census Division'].unique().size)
    
    eta = (beta_0 +
           alpha_race_gender[race_gender] + 
           alpha_age_gender[age_gender] +
           alpha_age_edu[age_edu] + 
           alpha_edu_gender[edu_gender] + 
           alpha_age[age] +
           alpha_edu[edu] +
           alpha_race[race] + 
           alpha_gender[gender] + 
           alpha_area[area] + 
           alpha_div[div])


In [12]:
with model:
    p = pm.math.sigmoid(eta)
    obs = pm.Binomial('obs', n, p,
                      observed=index1_unique['Will Vote'].values)


In [ ]:
with model:
    trace = pm.sample(draws=1000,
                      nuts_kwargs={'target_accept': 0.99},
                      init='advi',
                      njobs=6)

Auto-assigning NUTS sampler...
Initializing NUTS using advi...
Average Loss = 471.66:  15%|███████▍                                          | 29898/200000 [48:59<4:38:42, 10.17it/s]
Convergence archived at 29900
Interrupted at 29,899 [14%]: Average Loss = 590.67
C:\Anaconda3\envs\MSR\lib\site-packages\pymc3\model.py:384: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(var.dtype, float):
Multiprocess sampling (6 chains in 6 jobs)
NUTS: [sigma_division_log__, delta_division, sigma_area_log__, delta_area, sigma_gender_log__, delta_gender, sigma_race_log__, delta_race, sigma_education_log__, delta_education, sigma_age_log__, delta_age, sigma_education_gender_log__, delta_education_gender, sigma_age_education_log__, delta_age_education, sigma_age_gender_log__, delta_age_gender, sigma_race_gender_log__, delta_race_gender, beta_0, sigma_sta